# NLP - Corretor Ortográfico
Material baseado no curso **Corretor Ortográfico em Python: Aplicando técnicas de NLP** da **Alura**.

In [1]:
import nltk
import time

# Pacote do NLTK responsável pela transformação do texto em tokens válidos, separando a pontuação.
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\salatiel.bairros\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
arquivo_corpus = 'dados/artigos.txt'
arquivo_teste = "dados/palavras.txt"
alfabeto = 'abcdefghijklmnopqrstuvwxyzáàãâéèêíìóòõôúùûç'
corpus = []
probabilidades = {}
total_palavras = 0

In [3]:
def separa_palavras(lista_tokens):
    lista_palavras = []
    for token in lista_tokens:
        if token.isalpha():
            lista_palavras.append(token.lower())
    return lista_palavras

In [4]:
def carregar_corpus():
    global corpus
    global total_palavras
    global probabilidades
    
    with open(arquivo_corpus, 'r', encoding="utf8") as f:
        artigos = f.read()
        
    lista_palavras = separa_palavras(nltk.tokenize.word_tokenize(artigos))
    corpus = list(set(lista_palavras))
    total_palavras = len(lista_palavras)
    frequencia = nltk.FreqDist(corpus)
    for p in frequencia:
        probabilidades[p] = frequencia[p] / total_palavras

In [5]:
def gerar_fatias(palavra):
    fatias = []
    for i in range(len(palavra)+1):
        fatias.append((palavra[:i], palavra[i:]))
    return fatias

In [6]:
def insere_letras(fatias):    
    palavras_geradas = []
    for E, D in fatias:
        for letra in alfabeto:
            palavras_geradas.append(E + letra + D)
    return palavras_geradas

In [7]:
def deletando_caracteres(fatias):
    novas_palavras = []
    for E, D in fatias:
        novas_palavras.append(E + D[1:])
    return novas_palavras

In [8]:
def inverte_letra(fatias):
    novas_palavras = []
    for E, D in fatias:
        if len(D) > 1:
            novas_palavras.append(E + D[1] + D[0] + D[2:])
    return novas_palavras

In [9]:
def troca_letra(fatias):
    novas_palavras = []
    for E, D in fatias:
        for letra in alfabeto:
            novas_palavras.append(E + letra + D[1:])
    return novas_palavras

In [10]:
def gerador_palavras(palavra):
    fatias = gerar_fatias(palavra)
    palavras = insere_letras(fatias)
    palavras += deletando_caracteres(fatias)
    palavras += troca_letra(fatias)
    palavras += inverte_letra(fatias)
    return palavras

In [11]:
def obter_palavras_validas(palavras_possiveis):
    return [p for p in palavras_possiveis if p in corpus]

In [12]:
def corretor_bruto(palavra):
    return obter_palavras_validas(gerador_palavras(palavra))

In [13]:
def probabilidade(palavra_gerada):
    return probabilidades[palavra_gerada]

In [14]:
def corretor(palavra):
    palavras_geradas = corretor_bruto(palavra)
    if(len(palavras_geradas) == 0):
        return palavra
    palavra_correta = max(palavras_geradas, key = probabilidade)
    return palavra_correta

In [15]:
def validacao_individual():
    print('Inserindo:' + corretor('lgica'))
    print('Invertendo: ' + corretor('lóigca'))
    print('Trocando: ' + corretor('lpgica'))
    print('Removendo: ' +  corretor('lógiica'))

## Validando o corretor

In [16]:
def cria_dados_teste():
    lista_palavras_teste = []
    with open(arquivo_teste, "r", encoding="utf8") as f:
        for linha in f:
            conjunto = linha.split()
            lista_palavras_teste.append((conjunto[0], conjunto[1]))
    return lista_palavras_teste

In [17]:
def calcular_taxa_maxima(lista_teste):
    nao_existe = []
    for pC, pE in lista_teste:
        if (pC not in corpus):
            nao_existe.append(pC)
    erro_minimo = round((len(nao_existe) / len(corpus)) * 100, 2)
    print('% de palavras não existentes no vocabulário: ', erro_minimo)
    return nao_existe

In [18]:
def incrementar_vocabulario(ausentes):
    global total_palavras
    for p in ausentes:
        if(p not in corpus):
            total_palavras += 1
            probabilidades[p] = 1 / total_palavras
            corpus.append(p)

In [19]:
def avaliador (testes, funcCorretor):
    numero_palavras = len(testes)
    acertou = 0
    erradas = []
    start = time.time()
    index = 0
    pAtual = 0
    nTestes = len(testes)
    for correta, errada in testes:
        index += 1
        palavra_corrigida = funcCorretor(errada)
        if palavra_corrigida == correta:
            acertou += 1
        else:
            erradas.append((correta, errada, palavra_corrigida))
        new_p = int(round(100 * (index / nTestes), 0))
        if(new_p > pAtual):
            pAtual = new_p
            print(f'{pAtual:03}% ' + "*" * pAtual)      
    end = time.time()            
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    print(f"{taxa_acerto}% de {numero_palavras} palavras, executado em {end - start}.")
    return erradas

In [20]:
def gerador_turbinado(palavras_geradas):
    novas_palavras = []
    for palavra in palavras_geradas:
        for p in gerador_palavras(palavra):
            if(p in corpus):
                novas_palavras.append(p)
    return novas_palavras

In [24]:
def corretor_turbinado(palavra):
    palavras_geradas = corretor_bruto(palavra)
    if(len(palavras_geradas) == 0):
        palavras_geradas = gerador_turbinado(palavras_geradas)
        if(len(palavras_geradas) == 0):
            return palavra
    return max(palavras_geradas, key = probabilidade)

In [25]:
def executar():
    print("\n*** CORRETOR EM 3 CAMADAS ***\n")
    print("\n 1 - Carregando corpus\n")
    carregar_corpus()
    print("\n 2 - Corpus carregado\n")
    print("\n 3 - Avaliação individual\n")
    validacao_individual()
    print("\n 4 - Criando dados de teste\n")
    palavras_teste = cria_dados_teste()
    print("\n 5 - Calculando taxa mínima\n")
    ausentes = calcular_taxa_maxima(palavras_teste)
    print("\n 6 - Atualizando vocabulario\n")
    incrementar_vocabulario(ausentes)
    print("\n 7 - Iniciando avaliação\n")
    erradas = avaliador(palavras_teste, corretor_turbinado)
    print("\n 8 - Avaliação finalizada. Palavras erradas: \n")
    print(erradas)

In [26]:
executar()


*** CORRETOR EM 3 CAMADAS ***


 1 - Carregando corpus


 2 - Corpus carregado


 3 - Avaliação individual

Inserindo:lógica
Invertendo: lógica
Trocando: lógica
Removendo: lógica

 4 - Criando dados de teste


 5 - Calculando taxa mínima

% de palavras não existentes no vocabulário:  0.07

 6 - Atualizando vocabulario


 7 - Iniciando avaliação

001% *
002% **
003% ***
004% ****
005% *****
006% ******
007% *******
008% ********
009% *********
010% **********
011% ***********
012% ************
013% *************
014% **************
015% ***************
016% ****************
017% *****************
018% ******************
019% *******************
020% ********************
021% *********************
022% **********************
023% ***********************
024% ************************
025% *************************
026% **************************
027% ***************************
028% ****************************
029% *****************************
030% ******************************
031% *